# Import of Libraries

In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_trf")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
import spacy
import spacy_transformers
import xlrd
import re
import nltk
from spacy import displacy
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')
words=set(nltk.corpus.words.words())
nlp = spacy.load('en_core_web_trf')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Cleaning

Lemmatisation Function

In [ ]:
def lemmatize_text(tl):
    wnl = WordNetLemmatizer()
    lemmatizer_sentence = []
    tokenizer= nltk.tokenize.WhitespaceTokenizer()
    for word in tokenizer.tokenize(tl):
        lemmatizer_sentence.append(wnl.lemmatize(word,'v'))
        lemmatizer_sentence.append(" ")

    return("".join(lemmatizer_sentence))

Contraction Function

In [ ]:
def ex_contractions(tl):
    result = []
    for word in tl:
        # replace contracting withoutsignal
        word = word.replace("wont", "won't")
        word = word.replace("cant", "can't")
        word = word.replace("its", "it's")
        word = word.replace("youre", "you're")
        word = word.replace("hes", "he's")
        word = word.replace("shes", "she's")
        word = word.replace("its", "it's")
        word = word.replace("weare", "we're")
        word = word.replace("theyre", "they're")
        # specific
        word = re.sub(r"won\'t", "will not", str(word))
        word = re.sub(r"can\'t", "can not", str(word))
        # general
        word = re.sub(r"n\'t", " not", str(word))
        word = re.sub(r"\'re", " are", str(word))
        word = re.sub(r"\'s", " is", str(word))
        word = re.sub(r"\'d", " would", str(word))
        word = re.sub(r"\'ll", " will", str(word))
        word = re.sub(r"\'t", " not", str(word))
        word = re.sub(r"\'ve", " have", str(word))
        word = re.sub(r"\'m", " am", str(word))
        result.append(word)
    return result

Removal of Special Character Function

In [ ]:
def ex_specialChar(tl):
     return [re.sub('[^A-Za-z0-9]+', "", word) for word in tl]

Data Cleaning

In [ ]:
def data_cleaning(sentence):
    lemmatize_text(str(sentence))
    tl = str(sentence).split()
    if tl[0].isalpha():
        tl[0] = tl[0][0].lower() + tl[0][1:]
    tl = ex_contractions(tl)
    tl = ex_specialChar(tl)
    return tl

# Read and Write clean data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/tiktok.csv')
df.head()
comments = df["comment"]

cleaned_comments = []

# Loop through the comments to perform data cleaning
for comment in comments:
    clean_text = ' '.join(data_cleaning(str(comment)))
    clean_text = clean_text.lower()
    if len(clean_text) >=5:
      cleaned_comments.append(clean_text)

# Create a new DataFrame with the cleaned comments
new_df = pd.DataFrame({'cleaned_comments': cleaned_comments})
new_df

# Write the new DataFrame to a new CSV file in the same folder
new_df.to_csv('/content/drive/MyDrive/Datasets/CleanInsta.csv', index=False)

# Main Function

In [ ]:
# For ACD (categories)
synonyms = {
    "Quality": ["quality", "taste", "texture", "flavor", "marbling", "freshness", "juiciness", "tenderness", "aroma", "seasoning", "cut", "grade", "origins"],
    "Price": ["price", "cost", "value", "discount", "deal", "promotion", "offers", "pricing strategy", "expensive", "cheap", "affordable", "luxury"],
    "Packaging": ["packaging", "presentation", "wrap", "box", "label", "seal", "expiration date", "storage", "recyclable", "eco-friendly"],
    "Delivery": ["delivery", "shipping", "timing", "speed", "courier", "reliability", "tracking", "packaging", "delivery fee", "shipment"],
    "Customer Service": ["customer service", "staff", "helpful", "responsive", "knowledgeable", "friendly", "courteous", "assistance", "wait time", "communication"],
    "Availability": ["availability", "stock", "inventory", "supply chain", "scarce", "seasonal", "limited edition", "backorder", "pre-order", "in-stock"],
    "Culinary Experience": ["dining", "recipe", "cooking", "grilling", "marination", "pairing", "cuisine", "menu", "dish", "serving", "chef", "culinary"],
    "Ethical Considerations": ["ethics", "animal welfare", "sustainable", "local", "farming practices", "transparency", "certification", "traceability"],
    "Nutrition": ["nutrition", "health", "calories", "protein", "fat content", "nutritional value", "organic", "hormones", "additives"],
    "Brand": ["brand", "reputation", "reviews", "rating", "trustworthiness", "market presence", "awareness", "loyalty", "branding"],
}

Data Visualisation

In [ ]:
doc = nlp("I prefer those vegan beef substitutes to be cheaper so more people can afford them")
sentence_spans = list(doc.sents)
displacy.render(sentence_spans, style='dep', jupyter=True, options={'distance': 90})

In [ ]:
def capture_op(child, amod):
    if child.dep_ == 'amod' or child.dep_ == 'compound':
        amod.append(child.text)
        if check_negation(child):
            amod.append('not')

def capture_noun_dep(child, amod):
    if child.dep_ in ['acomp', 'advmod', 'amod']:
        amod.append(child.text)
        if check_negation(child):
            amod.append('not')

def parent_capture(parent, amod):
    for child in parent.children:
        if child.dep_ in ['acomp', 'advmod', 'amod']:
            amod.append(child.text)
            if check_negation(child):
                amod.append('not')

# Add this function to handle negation
def check_negation(token):
    for child in token.children:
        if child.dep_ == 'neg':
            return True
    return False

rawText = "Wagyu beef is bad"
doc = nlp(rawText)
sentences = list(doc.sents)
pairing = []

blob = TextBlob(rawText)
senti = blob.sentiment.polarity

for sentence in sentences:
    sentence = ' '.join(data_cleaning(str(sentence)))
    for token in nlp(sentence):
        if token.pos_ in ["NOUN", "PROPN"]:
            cat = ''
            for k, v in synonyms.items():
                if token.text.lower() in v:
                    cat = k
            target = token.text
            amod = []
            for child in token.children:
                capture_op(child, amod)
                capture_noun_dep(child, amod)
            for parent in token.ancestors:
                parent_capture(parent, amod)
            if amod:
                pairing.append({'category': cat, 'aspect': target, 'opinion': amod, 'sentiment': senti})

for pair in pairing:
    print(pair)

{'category': '', 'aspect': 'wagyu', 'opinion': ['bad'], 'sentiment': -0.6999999999999998}
{'category': '', 'aspect': 'beef', 'opinion': ['wagyu', 'bad'], 'sentiment': -0.6999999999999998}
